In [1]:
import sys
sys.path.append('../../..')

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import feature_column as fc
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn import preprocessing
from tensorflow.keras.models import save_model,load_model
from utils.preprocessing import *
from utils.dataset import Dataset
import pickle

from utils.evaluate import calculate_ctr, compute_rce, average_precision_score
import tensorflow.keras.backend as K
import core.config as conf

## Load Data & Preprocessing

In [14]:
ds = Dataset(train=True, target_encoding=False)

In [15]:
data_path = conf.dataset_mini_path + 'train'
ori_df = read_data(data_path)

In [4]:
ori_df.head()

,text_tokens,hashtags,tweet_id,media,links,domains,tweet_type,language,tweet_timestamp,creator_id,...,engager_id,engager_follower_count,engager_following_count,engager_is_verified,engager_account_creation,creator_follows_engager,reply_timestamp,retweet_timestamp,comment_timestamp,like_timestamp
0,101\t147\t69849\t11447\t57277\t63310\t136\t106...,NaN,6238B9E15E83B6D477394E9D80B3784E,Photo\tPhoto,NaN,NaN,TopLevel,7F4FAB1EB12CD95EDCD9DB2A6634EFCE,1612883086,F09233A58769507FD4E6FD618BCFA5B6,...,CFFD4D0D3AAF28B5397A98E1F4C92C77,595,1424,False,1307991260,False,NaN,NaN,NaN,NaN
1,101\t56898\t137\t10799\t11039\t168\t26978\t131...,NaN,731FB90C6CFEF1B71D322106573F71DB,Photo,NaN,NaN,Retweet,E7F038DE3EAD397AEC9193686C911677,1612864776,46BEEE566BB708F18075A60FDDAA8503,...,E0F9A3BB331C7B613B165F32F8D273BE,514,416,False,1336861089,True,NaN,NaN,NaN,NaN
2,101\t13304\t179\t14902\t74788\t10390\t33993\t1...,NaN,89276E5272498E858EE8AF691EBF0951,NaN,NaN,NaN,TopLevel,1F73BB863A39DB62B4A55B7E558DB1E8,1613661020,83F59BF2E0778AC4078FA3F7B71F5960,...,58ABF7FB24D65FB94C6487915A889460,376,311,False,1602935552,True,NaN,NaN,NaN,1.613662e+09
3,101\t620\t55399\t17451\t69304\t82692\t572\t105...,NaN,502FDBC0EB4E7AB157D38262817716EB,NaN,F65FE23F3E97EE91DB990B4E02FC8DFC,28F197E209F61EE2F1C97FBF9128976D,TopLevel,310ECD7D1E42216E3C1B31EFDDFC72A7,1612751908,C96110509F8B6256CE08A8AE85ED75DB,...,76730357A84D1D04A662FDB1449F1AFF,22461,16342,False,1366022767,True,NaN,NaN,NaN,NaN
4,101\t10747\t12723\t10124\t29956\t91518\t10142\...,NaN,CF1F523F7D4D4139E5FD3EBD72F27D5A,NaN,NaN,NaN,TopLevel,488B32D24BD4BB44172EB981C1BCA6FA,1613060168,2031F18622E7C6A30E702D33776BF2C1,...,9008C5AB3CB46623ADBD4C68A2A2D088,388,647,False,1410270773,False,NaN,NaN,NaN,1.613061e+09


In [16]:
df = ds.preprocess(ori_df)

In [17]:
df

,creator_follows_engager,creator_account_creation,language,creator_id,creator_is_verified,engager_following_count,domains,engager_is_verified,media,engager_account_creation,...,engager_id,creator_following_count,reply,retweet,comment,like,dt_day,dt_dow,dt_hour,len_domains
0,0,1525021638,10,F09233A58769507FD4E6FD618BCFA5B6,0,1424,0,0,4,1307991260,...,CFFD4D0D3AAF28B5397A98E1F4C92C77,645,0.0,0.0,0.0,0.0,9,1,15,0
1,1,1348805977,1,46BEEE566BB708F18075A60FDDAA8503,0,416,0,0,1,1336861089,...,E0F9A3BB331C7B613B165F32F8D273BE,3690,0.0,0.0,0.0,0.0,9,1,9,0
2,1,1583939938,5,83F59BF2E0778AC4078FA3F7B71F5960,0,311,0,0,0,1602935552,...,58ABF7FB24D65FB94C6487915A889460,146,0.0,0.0,0.0,1.0,18,3,15,0
3,1,1306675315,12,C96110509F8B6256CE08A8AE85ED75DB,1,16342,1,0,0,1366022767,...,76730357A84D1D04A662FDB1449F1AFF,2124,0.0,0.0,0.0,0.0,8,0,2,1
4,0,1308377880,0,2031F18622E7C6A30E702D33776BF2C1,1,647,0,0,0,1410270773,...,9008C5AB3CB46623ADBD4C68A2A2D088,1101,0.0,0.0,0.0,1.0,11,3,16,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4338901,0,1359315180,1,968FAF2F52AACA58C2B41CE5B13A811D,0,1908,0,0,1,1420785574,...,1BC672DA2F063F2DE03DA89BA6B621C5,315,0.0,0.0,0.0,1.0,7,6,4,0
4338902,1,1293912925,0,54652625625729AE0E38196970594938,0,713,0,0,0,1311226875,...,39D3029E8FDC3A18472D28436EC9E776,937,0.0,0.0,0.0,0.0,8,0,0,0
4338903,0,1512095330,3,614E269F8E5D731946BDBC86E9DC4F4D,0,48,0,0,4,1597880960,...,4A4E526A7390A5CC9D02ECB5CB62F377,121,0.0,0.0,0.0,1.0,4,3,0,0
4338904,0,1251066468,0,5A5C7CCA69CDEBBD7E81C3797DE9DD7E,0,135,0,0,4,1604003943,...,D8CAFE1402CE86E007370664FC553C2B,368,0.0,0.0,0.0,1.0,10,2,11,0


In [19]:
pkl_path = conf.dict_path + 'user_main_language.pkl'
with open(pkl_path, 'rb') as f:
    user_to_main_language = pickle.load(f)

In [20]:
df['engager_main_language'] = df['engager_id'].apply(lambda x: user_to_main_language[x])
df['creator_main_language'] = df['creator_id'].apply(lambda x: user_to_main_language[x])

In [21]:
df['is_same_main_language'] = df['engager_main_language'] == df['creator_main_language']

In [22]:
df

,creator_follows_engager,creator_account_creation,language,creator_id,creator_is_verified,engager_following_count,domains,engager_is_verified,media,engager_account_creation,...,retweet,comment,like,dt_day,dt_dow,dt_hour,len_domains,engager_main_language,creator_main_language,is_same_main_language
0,0,1525021638,10,F09233A58769507FD4E6FD618BCFA5B6,0,1424,0,0,4,1307991260,...,0.0,0.0,0.0,9,1,15,0,35,9,False
1,1,1348805977,1,46BEEE566BB708F18075A60FDDAA8503,0,416,0,0,1,1336861089,...,0.0,0.0,0.0,9,1,9,0,1,1,True
2,1,1583939938,5,83F59BF2E0778AC4078FA3F7B71F5960,0,311,0,0,0,1602935552,...,0.0,0.0,1.0,18,3,15,0,4,5,False
3,1,1306675315,12,C96110509F8B6256CE08A8AE85ED75DB,1,16342,1,0,0,1366022767,...,0.0,0.0,0.0,8,0,2,1,1,12,False
4,0,1308377880,0,2031F18622E7C6A30E702D33776BF2C1,1,647,0,0,0,1410270773,...,0.0,0.0,1.0,11,3,16,0,13,0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4338901,0,1359315180,1,968FAF2F52AACA58C2B41CE5B13A811D,0,1908,0,0,1,1420785574,...,0.0,0.0,1.0,7,6,4,0,1,1,True
4338902,1,1293912925,0,54652625625729AE0E38196970594938,0,713,0,0,0,1311226875,...,0.0,0.0,0.0,8,0,0,0,0,0,True
4338903,0,1512095330,3,614E269F8E5D731946BDBC86E9DC4F4D,0,48,0,0,4,1597880960,...,0.0,0.0,1.0,4,3,0,0,3,17,False
4338904,0,1251066468,0,5A5C7CCA69CDEBBD7E81C3797DE9DD7E,0,135,0,0,4,1604003943,...,0.0,0.0,1.0,10,2,11,0,0,0,True


In [23]:
df['retweet']  = df['retweet'].astype(np.bool)
df['like'] = df['like'].astype(np.bool)
df['comment'] = df['comment'].astype(np.bool)
df['reply'] = df['reply'].astype(np.bool)

In [24]:
df['is_positive'] = df['retweet'] | df['like'] | df['comment'] | df['reply']

In [25]:
positive = df[df['is_positive'] == True]

In [26]:
len(positive)

2164208

In [27]:
cnt_same_main_language = len(positive[positive['is_same_main_language']==True])
cnt_different_main_language = len(positive[positive['is_same_main_language']==False])

In [28]:
cnt_same_main_language

817233

In [29]:
cnt_different_main_language

1346975

In [32]:
positive[positive['is_same_main_language']==False][['engager_main_language', 'creator_main_language']].tail(50)

,engager_main_language,creator_main_language
4338734,35,17
4338737,9,3
4338739,2,3
4338741,2,9
4338744,0,2
4338746,11,24
4338748,35,24
4338751,1,35
4338757,9,35
4338759,2,1
